In [1]:
import numpy as np
from skyfield.api import load, wgs84, Distance
from skyfield.toposlib import ITRSPosition
from skyfield.framelib import itrs
from coverage_v2 import *
from scipy.spatial.transform import Rotation as R

import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
tles = gen_sats(sat_nos=[48915])
sat = tles[0][0]
times = gen_times(start_yr=2021, start_mo=11, start_day=27, days=1, step_min=1)

hfov_x = 15
hfov_y = 10


Satellite(s) Loaded from TLE:
[<EarthSatellite YAM-3 catalog #48915 epoch 2021-11-27 14:32:26 UTC>]
Propogation time: 
 2021-11-27 00:00:00+00:00 
to 
 2021-11-27 23:59:00+00:00


In [3]:
# https://github.com/skyfielders/python-skyfield/issues/492
# https://github.com/skyfielders/python-skyfield/issues/557

# lvlh issues

In [4]:
cs_dict = {"c1": {"X" : -hfov_x, "Y": hfov_y}, 
    "c2": {"X" : hfov_x, "Y": hfov_y},
    "c3": {"X" : hfov_x, "Y": -hfov_y},
    "c4": {"X" : -hfov_x, "Y": -hfov_y}
}

cs_lla_dict = {"c1": {"lat" : None, "lon": None}, 
    "c2": {"lat" : None, "lon": None},
    "c3": {"lat" : None, "lon": None},
    "c4": {"lat" : None, "lon": None}
}

In [5]:
polys = []

for time in times:

    SSP_lat, SSP_lon, SSP_d, lvlh, vec = get_los2(sat, time)
    xyz_rates = sat.at(time).frame_xyz_and_velocity(itrs)
    xyz_dist = xyz_rates[0]

    for c in cs_dict:
        # Generate X anx Y rotation vectors
        rot_X_deg = cs_dict[c]["X"]
        rot_X_rad = np.radians(rot_X_deg)
        rot_X_ax = lvlh["X"]

        rot_Y_deg = cs_dict[c]["Y"]
        rot_Y_rad = np.radians(rot_Y_deg)
        rot_Y_ax = lvlh["Y"]

        # Rotate about X
        rot_X_vec = rot_X_rad * rot_X_ax
        rot_X = R.from_rotvec(rot_X_vec)
        los_X = rot_X.apply(vec)

        # Rotate about Y
        rot_Y_vec = rot_Y_rad * rot_Y_ax
        rot_Y = R.from_rotvec(rot_Y_vec)
        los_XY = rot_Y.apply(los_X)

        los_xyz = los_to_earth(xyz_dist.km, los_XY)
        los = Distance(km=los_xyz)
        los_itrs = ITRSPosition(los)
        los_itrs.at(time).frame_xyz(itrs).km

        los_lat, los_lon = wgs84.latlon_of(los_itrs.at(time))
        cs_lla_dict[c]["lat"] = los_lat.degrees
        cs_lla_dict[c]["lon"] = los_lon.degrees

    polys.append(Polygon([(cs_lla_dict["c1"]["lon"], cs_lla_dict["c1"]["lat"]), 
            (cs_lla_dict["c2"]["lon"], cs_lla_dict["c2"]["lat"]), 
            (cs_lla_dict["c3"]["lon"], cs_lla_dict["c3"]["lat"]),
            (cs_lla_dict["c4"]["lon"], cs_lla_dict["c4"]["lat"]),
            (cs_lla_dict["c1"]["lon"], cs_lla_dict["c1"]["lat"])]
            # (gdf.loc[n].bound1_lon, gdf.loc[n].bound1_lat)
            ))
        

In [6]:
cs_lla_dict

{'c1': {'lat': -17.2115198606779, 'lon': 35.41217205481796},
 'c2': {'lat': -17.733062003113368, 'lon': 32.718345675069415},
 'c3': {'lat': -19.411265435401354, 'lon': 33.06063443730512},
 'c4': {'lat': -18.884055435509453, 'lon': 35.78116788900032}}

In [7]:
poly_df = gpd.GeoDataFrame(
        data=polys, 
        columns=['geometry'], 
        crs="EPSG:4326"
        )

In [8]:
poly_plot = poly_df.geometry.to_crs("ESRI:54009")
poly_plot = poly_plot[poly_plot.area < poly_plot.area.quantile(.98)]
poly_plot.explore()